In [1]:
import pandas as pd
import pyNetLogo
from SALib.sample import saltelli
from SALib.analyze import sobol



In [2]:
netlogo = pyNetLogo.NetLogoLink(gui=True, jvm_home = r"C:\Program Files\Java\jdk-15.0.1\bin\server\jvm.dll")


In [3]:
netlogo.load_model(r"C:\Users\bruno\github\SEN1211_project\models\final\model\final.nlogo")
netlogo.command('setup')

In [4]:
#netlogo.command('repeat 1000 [go]')

In [18]:
problem = {
  'num_vars': 6,
  'names': ['agents-at-start',
            'percentage-female',
            'percentage-children',
            'percentage-visitors-go-to-preferred-exit',
            'percentage-stationary-staff',
            'alerting-range'],
  'bounds': [[450., 451.],
             [37., 38.],
             [5., 6.],
             [96., 97.],
             [50., 51.],
             [6., 7.]]
}



In [26]:
problem = {
  'num_vars': 6,
  'names': ['agents-at-start'],
  'bounds': [[450., 451.]]
}


In [30]:
param_values = saltelli.sample(problem, 1, calc_second_order=True)


In [31]:
param_values

array([[450.21972656, 450.09667969, 450.51855469, 450.67675781,
        450.28027344, 450.90722656],
       [450.04589844, 450.09667969, 450.51855469, 450.67675781,
        450.28027344, 450.90722656],
       [450.21972656, 450.89941406, 450.51855469, 450.67675781,
        450.28027344, 450.90722656],
       [450.21972656, 450.09667969, 450.50097656, 450.67675781,
        450.28027344, 450.90722656],
       [450.21972656, 450.09667969, 450.51855469, 450.06933594,
        450.28027344, 450.90722656],
       [450.21972656, 450.09667969, 450.51855469, 450.67675781,
        450.08496094, 450.90722656],
       [450.21972656, 450.09667969, 450.51855469, 450.67675781,
        450.28027344, 450.25488281],
       [450.21972656, 450.89941406, 450.50097656, 450.06933594,
        450.08496094, 450.25488281],
       [450.04589844, 450.09667969, 450.50097656, 450.06933594,
        450.08496094, 450.25488281],
       [450.04589844, 450.89941406, 450.51855469, 450.06933594,
        450.08496094, 450.2

In [32]:
def simulation(experiment):

    #Set the input parameters
    netlogo.command('setup')
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            print('set {0} {1}'.format(name, experiment[i]))
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    counts = netlogo.repeat_report(['count turtles','count visitors'], 100)
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    

    results = pd.Series([counts['count turtles'].values.mean(),
                         counts['count visitors'].values.mean()],
                         index=['Avg. turtles', 'Avg. visitors'])

    return counts

In [33]:
results1 = pd.DataFrame(simulation(param_values))

0 agents-at-start
set agents-at-start [450.21972656 450.09667969 450.51855469 450.67675781 450.28027344
 450.90722656]


In [42]:
netlogo.command('set agents-at-start 450')

In [35]:
results1

count turtles count visitors
0.0         5000.0         4950.0
1.0         4999.0         4949.0
2.0         4999.0         4949.0
3.0         4997.0         4947.0
4.0         4997.0         4947.0
...            ...            ...
95.0        4760.0         4712.0
96.0        4752.0         4704.0
97.0        4747.0         4699.0
98.0        4734.0         4686.0
99.0        4723.0         4675.0

[100 rows x 2 columns]

In [34]:
param_values[1]

array([450.04589844, 450.09667969, 450.51855469, 450.67675781,
       450.28027344, 450.90722656])

In [11]:
#Si = sobol.analyze(problem, results['count turtles'].values, calc_second_order=True, print_to_console=False)